# Musical notes and intervals, consonances and dissonances, temperaments

Code to explore dyadic intervals and their degree of consonance, triads and tetrads, as well as different temperaments.

In [6]:
from math import log, exp
import numpy as np
import scipy.signal as ss
import matplotlib.pyplot as plt
from matplotlib.ticker import LogFormatter, FormatStrFormatter
from ipywidgets import interact, interactive_output
from IPython.display import display as wdisplay
import ipywidgets as widgets
import harmony
# $matplotlib

In [7]:
# initialization
plt.rc('figure', figsize=(20, 10))
harmony.init()
k_harp = 0.45
midiin = None

# roughness model: this is the CR_alfa with mistuned 8th and 5th data computed with Mathematica and exported to a file
harmony.roughness[1.0], harmony.roughness[1.1], harmony.roughness[2.0]

(0.9872312095616033, 0.08983148809661412, 0.9836321050882914)

In [8]:
# Prepare UI
def leftlayout(degree):
    return widgets.Layout(left='-%d%%' % degree)

def genUI_P():
    w_phys_label = widgets.HTML(value='<b><center>Physical parameters</center></b>')
    w_duration = widgets.FloatSlider(min=0.1, max=6, value=2, step=0.1, continuous_update=False, description='Duration [s]')
    w_harmonics = widgets.IntSlider(min=1, max=20, value=5, continuous_update=False, description='no. Harmonics',
                                    style={'description_width': 'initial'})
    w_h_model = widgets.RadioButtons(options=harmony.h_models, description='Harm. model')
    w_k_harp = widgets.FloatSlider(min=0.01, max=0.5, value=k_harp, step=0.01, continuous_update=False, description='k_harp')
    w_addcb = widgets.Checkbox(description='Add common bass')
    w_miscb = widgets.IntSlider(min=0, max=5, value=0, continuous_update=False, description='Mistuned c.b. [Hz]',
                                style={'description_width': 'initial'})
    w_miscb.layout.visibility = 'hidden'
    w_k_harp.layout.visibility = 'hidden'
    w_spectrogram = widgets.Checkbox(description='Show Spectrogram')

    def handle_h_model(change):
        w_k_harp.layout.visibility = 'visible' if change['new'] == 'harp' else 'hidden'

    def handle_addcb(change):
        w_miscb.layout.visibility = 'visible' if change['new'] else 'hidden'

    w_h_model.observe(handle_h_model, names='value')
    w_addcb.observe(handle_addcb, names='value')

    return widgets.VBox([w_phys_label, w_duration, w_harmonics, w_h_model, w_k_harp, w_addcb, w_miscb, w_spectrogram],
                        layout=leftlayout(20)), {
        'duration': w_duration,
        'harmonics': w_harmonics,
        'h_model': w_h_model,
        'k_harp': w_k_harp,
        'addcb': w_addcb,
        'miscb': w_miscb,
        'spectrogram': w_spectrogram,
    }

def genUI_M():
    w_mus_label = widgets.HTML(value='<b>&nbsp; &nbsp; &nbsp; Musical parameters</b>')
    w_temperament = widgets.RadioButtons(options=harmony.temperaments, description='Temperament')
    w_tonic = widgets.IntSlider(min=-11, max=62, value=0, continuous_update=False, description='Tonic')
    w_interval = widgets.IntSlider(min=0, max=62, value=7, continuous_update=False, description='Interval')
    w_c_int2 = widgets.Checkbox(description='Interval 2', layout=leftlayout(20))
    w_interval2 = widgets.IntSlider(min=1, max=62, value=4, continuous_update=False, layout=leftlayout(30))
    return widgets.VBox([w_mus_label, w_temperament, w_tonic, w_interval,
                         widgets.HBox([w_c_int2, w_interval2], layout=widgets.Layout(width='70%', align_content='stretch'))
                       ]), {
        'temperament': w_temperament,
        'tonic': w_tonic,
        'int1': w_interval,
        'c_int2': w_c_int2,
        'int2': w_interval2
       }

def genUI_MC12():
    w_mus_label = widgets.HTML(value='<b><center>Musical parameters</center></b>', layout=leftlayout(30))
    w_temperament12 = widgets.RadioButtons(options=[t for t in harmony.temperaments if '12' in t], description='Temperament')
    w_tonic = widgets.IntSlider(min=-11, max=11, value=0, continuous_update=False, description='Tonic')
    w_c_III = widgets.Checkbox(description='III', layout=leftlayout(12))
    w_III = widgets.IntSlider(min=2, max=5, value=4, continuous_update=False, layout=leftlayout(40))
    w_c_V = widgets.Checkbox(description='V', layout=leftlayout(12))
    w_V = widgets.IntSlider(min=5, max=8, value=7, continuous_update=False, layout=leftlayout(40))
    w_c_VII = widgets.Checkbox(description='VII', layout=leftlayout(12))
    w_VII = widgets.IntSlider(min=8, max=12, value=10, continuous_update=False, layout=leftlayout(40))
    w_c_IX = widgets.Checkbox(description='IX', layout=leftlayout(12))
    w_IX = widgets.IntSlider(min=13, max=19, value=14, continuous_update=False, layout=leftlayout(40))
    return widgets.VBox([w_mus_label, w_temperament12, w_tonic, widgets.HBox([w_c_III, w_III]),
                         widgets.HBox([w_c_V, w_V]), widgets.HBox([w_c_VII, w_VII]), widgets.HBox([w_c_IX, w_IX])
                        ]), {
        'temperament': w_temperament12,
        'tonic': w_tonic,
        'c_III': w_c_III,
        'III': w_III,
        'c_V': w_c_V,
        'V': w_V,
        'c_VII': w_c_VII,
        'VII': w_VII,
        'c_IX': w_c_IX,
        'IX': w_IX
    }

def genUI_MC():
    w_mus_label = widgets.HTML(value='<b><center>Musical parameters</center></b>', layout=leftlayout(20))
    w_temperament = widgets.RadioButtons(options=harmony.temperaments, description='Temperament')
    w_tonic = widgets.IntSlider(min=-11, max=62, value=0, continuous_update=False, description='Tonic')
    w_interval = widgets.IntSlider(min=0, max=62, value=7, continuous_update=False, description='Interval')
    w_chord = widgets.RadioButtons(options=harmony.ChordsCollection.chords, value='maj1',
                                   description='Chord', layout=leftlayout(15))
    return widgets.VBox([w_mus_label, widgets.HBox([widgets.VBox([w_temperament, w_tonic]), w_chord])
                        ]), {
        'temperament': w_temperament,
        'tonic': w_tonic,
        'chord': w_chord
    }

def genUI_MMidi():
    w_mus_label = widgets.HTML(value='<b><center>Musical parameters</center></b>')
    w_temperament = widgets.RadioButtons(options=harmony.temperaments, description='Temperament')
    return widgets.VBox([w_mus_label, w_temperament]), {
        'temperament': w_temperament,
    }

w_out = widgets.Output(layout=widgets.Layout(width='75%', left='25%'))
@w_out.capture()
def captureoutput(obj, method, *args):
    w_out.clear_output(wait=True)
    return getattr(obj, method)(*args)

## Dyads, triads, and more standard chords

In [9]:
def interactive_dyads_triads(temperament, tonic, int1, c_int2, int2, duration, harmonics,
                             h_model, k_harp, addcb, miscb, spectrogram):
    if not addcb:
        miscb = 0
    elif miscb == 0:
        miscb = -1
    c = harmony.IChord(temperament, tonic, duration, harmonics, False, h_model, k_harp, miscb)
    # generate signal for the given interval
    if int1 == 0:
        s = c.note(tonic)
        d = 0
    elif not c_int2:
        s, d = captureoutput(c, 'dichord', int1)
    else:
        s, d = captureoutput(c, 'triad', int1, int2)
    c.play(s)
    c.plottimefreq(s, spectrogram)

    #c.save(s, 'interval%02d.wav' % tonic)     # harmonics = 10, h_model = harp-verotta for the test sounds
    # wait for playback to finish before exiting
    #p.wait_done()

uiM = genUI_M()
uiP = genUI_P()
ui = widgets.VBox([widgets.HBox([uiM[0], uiP[0]], layout=widgets.Layout(justify_content='center')), w_out])
interactive = interactive_output(interactive_dyads_triads, uiM[1] | uiP[1])
wdisplay(ui, interactive)

Output()

In [5]:
def interactive_standard_12chords(temperament, tonic, c_III, III, c_V, V, c_VII, VII, c_IX, IX,
                                  duration, harmonics, h_model, k_harp, addcb, miscb, spectrogram):
    if not addcb:
        miscb = 0
    elif miscb == 0:
        miscb = -1
    c = harmony.IChord(temperament, tonic, duration, harmonics, False, h_model, k_harp, miscb)
    # generate signal for the given chord
    intervals = []
    if c_III:
        intervals += [III]
    if c_V:
        intervals += [V]
    if c_VII:
        intervals += [VII]
    if c_IX:
        intervals += [IX]
    if len(intervals) > 0:
        s, d = captureoutput(c, 'cluster', intervals)
    else:
        s = c.note(tonic)
        d = 0
    c.play(s)
    c.plottimefreq(s, spectrogram)

    # wait for playback to finish before exiting
    #p.wait_done()

uiM = genUI_MC12()
uiP = genUI_P()
ui = widgets.VBox([widgets.HBox([uiM[0], uiP[0]], layout=widgets.Layout(justify_content='center')), w_out])
interactive = interactive_output(interactive_standard_12chords, uiM[1] | uiP[1])
wdisplay(ui, interactive)

Output()

In [ ]:
# The support for >12-tone temperaments is limited
def interactive_named_chords(temperament, tonic, chord, duration, harmonics, h_model, k_harp, addcb, miscb, spectrogram):
    if not addcb:
        miscb = 0
    elif miscb == 0:
        miscb = -1
    if spectrogram:
        duration = 1    # override to have a meaningful plot
    c = harmony.ChordsCollection(temperament, tonic, duration, harmonics, False, h_model, k_harp, addcb)
    # generate signal for the given chord
    s, d = captureoutput(c, chord)
    c.play(s)
    c.plottimefreq(s, spectrogram)

    # wait for playback to finish before exiting
    #p.wait_done()

uiM = genUI_MC()
uiP = genUI_P()
ui = widgets.VBox([widgets.HBox([uiM[0], uiP[0]], layout=widgets.Layout(justify_content='center')), w_out])
interactive = interactive_output(interactive_named_chords, uiM[1] | uiP[1])
wdisplay(ui, interactive)